<a href="https://colab.research.google.com/github/subh007/ml-ground/blob/main/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install tensorflow

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("robervalt/sunspots")

print("Path to dataset files:", path)

100%|██████████| 22.4k/22.4k [00:00<00:00, 18.4MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/robervalt/sunspots/versions/3


In [4]:
import pandas as pd

In [6]:
df = pd.read_csv(path + "/Sunspots.csv")

In [8]:
df.head()

,Unnamed: 0,Date,Monthly Mean Total Sunspot Number
0,0,1749-01-31,96.7
1,1,1749-02-28,104.3
2,2,1749-03-31,116.7
3,3,1749-04-30,92.8
4,4,1749-05-31,141.7
